# Job Queue Example with FlowerPower

**Execution:** `uvx --with "redis>=4.5.0,rq>=1.15.0,msgspec>=0.18.0,pandas>=2.0.0,matplotlib,seaborn" jupyter lab`

In [ ]:
# Setup and imports
import sys
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import random

# FlowerPower imports
from flowerpower.job_queue.rq import RQManager
from flowerpower.job_queue.rq.setup import RQBackend

# Import task functions
sys.path.append(str(Path.cwd() / "tasks"))
from data_processing import (
    simple_calculation,
    process_batch_data,
    generate_report,
    send_notification,
    cleanup_old_files,
    long_running_computation,
    data_validation_task,
)

print("🚀 Job Queue with FlowerPower")
print("=================================")

## 1. Initialize Job Queue Manager

In [ ]:
# Initialize FlowerPower JobQueueManager
try:
    # Create backend configuration
    backend = RQBackend(
        uri="redis://localhost:6379/0",
        queues=["default", "calculations", "reports", "notifications"],
    )

    # Initialize manager
    manager = RQManager(
        name="job_queue_example", base_dir=str(Path.cwd()), backend=backend
    )

    print("✅ JobQueueManager initialized")
    print(f"📋 Available queues: {', '.join(manager._queue_names)}")

except Exception as e:
    print(f"❌ Failed to initialize: {e}")
    print("💡 Make sure Redis is running: redis-server")
    manager = None

## 2. Enqueue Jobs

In [ ]:
# Enqueue various jobs
if manager:
    print("📥 Enqueuing Jobs")
    print("==================")

    jobs = []

    # Simple calculations
    print("\n🧮 Calculations:")
    for i in range(3):
        x, y = random.randint(1, 100), random.randint(1, 100)
        job = manager.enqueue(
            simple_calculation, x, y, operation="add", queue_name="calculations"
        )
        jobs.append(job)
        print(f"   • Job {job.id[:8]}: {x} + {y}")

    # Batch processing
    print("\n📊 Batch Processing:")
    data = [{"id": i, "value": random.randint(1, 100)} for i in range(20)]
    batch_job = manager.enqueue(
        process_batch_data, data, batch_size=5, queue_name="calculations"
    )
    jobs.append(batch_job)
    print(f"   • Job {batch_job.id[:8]}: process {len(data)} items")

    # Report generation
    print("\n📋 Reports:")
    for report_type in ["summary", "detailed"]:
        job = manager.enqueue(
            generate_report,
            f"data_source_{report_type}",
            report_type=report_type,
            queue_name="reports",
        )
        jobs.append(job)
        print(f"   • Job {job.id[:8]}: {report_type} report")

    # Notifications
    print("\n📧 Notifications:")
    for recipient in ["user@example.com", "admin@company.com"]:
        job = manager.enqueue(
            send_notification,
            recipient,
            "Test notification",
            channel="email",
            queue_name="notifications",
        )
        jobs.append(job)
        print(f"   • Job {job.id[:8]}: notify {recipient}")

    print(f"\n🎉 Enqueued {len(jobs)} jobs")

    # Show queue status
    print("\n📊 Queue Status:")
    for queue_name in manager._queue_names:
        count = len(manager._queues[queue_name])
        print(f"   • {queue_name}: {count} jobs")
else:
    print("❌ Cannot enqueue jobs - manager not initialized")
    jobs = []

## 3. Start Worker and Monitor Jobs

In [ ]:
# Start worker and monitor progress
if manager and jobs:
    print("👷 Starting Worker")
    print("==================")

    # Start worker in background
    manager.start_worker(background=True)

    # Monitor job progress
    print("\n👁️ Monitoring Jobs")
    print("==================")

    completed = []
    failed = []

    for round_num in range(10):
        print(f"\nRound {round_num + 1}/10:")

        status_counts = {"queued": 0, "started": 0, "finished": 0, "failed": 0}

        for job in jobs:
            try:
                job.refresh()
                status = job.get_status()
                status_counts[status] += 1

                if status == "finished" and job not in completed:
                    completed.append(job)
                elif status == "failed" and job not in failed:
                    failed.append(job)
            except:
                pass

        # Display status
        for status, count in status_counts.items():
            if count > 0:
                emoji = {
                    "queued": "⏳",
                    "started": "🔄",
                    "finished": "✅",
                    "failed": "❌",
                }[status]
                print(f"   {emoji} {status.title()}: {count}")

        # Check if all jobs are complete
        if status_counts["queued"] + status_counts["started"] == 0:
            print("\n🏁 All jobs completed!")
            break

        time.sleep(2)

    # Stop worker
    manager.stop_worker()
    print("\n✅ Worker stopped")
else:
    print("❌ Cannot monitor jobs - no jobs or manager not available")

## 4. View Results

In [ ]:
# Display job results
if completed:
    print("🎉 Completed Jobs")
    print("==================")

    for i, job in enumerate(completed[:5]):  # Show first 5
        print(f"\nJob {i + 1}: {job.id[:8]}")
        print(f"   Function: {job.func_name}")
        print(f"   Queue: {job.origin}")

        if job.result:
            if isinstance(job.result, dict):
                # Show key results
                for key, value in list(job.result.items())[:3]:
                    print(f"   {key}: {value}")
                if len(job.result) > 3:
                    print(f"   ... and {len(job.result) - 3} more fields")
            else:
                print(f"   Result: {str(job.result)[:100]}...")
else:
    print("⚠️ No completed jobs to display")

if failed:
    print("\n❌ Failed Jobs")
    print("==============")

    for job in failed:
        print(f"Job {job.id[:8]}: {job.exc_info or 'Unknown error'}")

## 5. Job Scheduling

In [ ]:
# Schedule future jobs
if manager:
    print("⏰ Scheduling Jobs")
    print("==================")

    scheduled_jobs = []

    # Schedule a job for 30 seconds from now
    future_time = datetime.now() + timedelta(seconds=30)
    scheduled_job = manager.enqueue_at(
        future_time,
        generate_report,
        "scheduled_data",
        report_type="summary",
        queue_name="reports",
    )
    scheduled_jobs.append(scheduled_job)
    print(
        f"• Scheduled job {scheduled_job.id[:8]} at {future_time.strftime('%H:%M:%S')}"
    )

    # Schedule a job with delay
    delayed_job = manager.enqueue_in(
        60,  # 60 seconds
        send_notification,
        "delayed@example.com",
        "Delayed notification",
        channel="email",
        queue_name="notifications",
    )
    scheduled_jobs.append(delayed_job)
    print(f"• Delayed job {delayed_job.id[:8]} in 60 seconds")

    # Start scheduler to process scheduled jobs
    print("\n🔄 Starting scheduler...")
    manager.start_scheduler(background=True)

    # Wait a bit and check scheduled jobs
    time.sleep(5)

    print("\n📋 Scheduled Jobs:")
    schedules = manager.get_schedules()
    for schedule in schedules:
        print(f"• {schedule.id[:8]}: {schedule.func_name}")

    # Stop scheduler
    manager.stop_scheduler()
    print("\n✅ Scheduler stopped")
else:
    print("❌ Cannot schedule jobs - manager not available")

## 6. Queue Analytics

In [ ]:
# Simple queue analytics
if manager:
    print("📊 Queue Analytics")
    print("==================")

    # Get all jobs from queues
    all_jobs = manager.get_jobs()

    for queue_name, queue_jobs in all_jobs.items():
        print(f"\n📋 Queue: {queue_name}")
        print(f"   Total jobs: {len(queue_jobs)}")

        # Count by status
        status_counts = {"queued": 0, "started": 0, "finished": 0, "failed": 0}
        for job in queue_jobs:
            try:
                status = job.get_status()
                status_counts[status] += 1
            except:
                status_counts["queued"] += 1

        for status, count in status_counts.items():
            if count > 0:
                print(f"   {status}: {count}")

    # Simple visualization
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))

    queue_names = list(all_jobs.keys())
    queue_lengths = [len(jobs) for jobs in all_jobs.values()]

    ax.bar(queue_names, queue_lengths, color="skyblue")
    ax.set_title("Jobs per Queue")
    ax.set_xlabel("Queue")
    ax.set_ylabel("Number of Jobs")

    plt.tight_layout()
    plt.show()
else:
    print("❌ Cannot analyze queues - manager not available")

## Next Steps

- Explore the [FlowerPower documentation](../../docs/) for advanced features
- Check the [CLI examples](../) for more usage patterns
- Review the [task functions](tasks/data_processing.py) for custom job implementations